# Circle dataset — Diffusion model inference (trained model)

This notebook loads a pre-trained Diffusion posterior sampler and performs inference on the circle dataset. Configure `load_path`, `epochs`, and `seed` to match your saved checkpoint names.


In [3]:
from pathlib import Path
import sys
# Project path setup
project_root = Path.cwd()
if (project_root / 'src').exists():
    pass
else:
    # Fallback: user-specific path
    project_root = Path('/home/chu034/Yaohang_Li/cDiff')


if not project_root.exists() or not (project_root / 'src').exists():
    raise FileNotFoundError(f"找不到项目目录或 src: {project_root}")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"项目路径: {project_root}")
print("Python 路径已更新")


项目路径: /home/chu034/Yaohang_Li/cDiff
Python 路径已更新


In [13]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from datasets.circle import return_circle_dl, set_circle_noise_sigma
from models.neural_sampler import DiffusionPosteriorSampler
from utils import load_torch_model, SET_SEED

# ----- Config -----
# Match these with your training run
load_path = "/home/chu034/Yaohang_Li/cDiff/result/circle"   # --save_path result --dataset circle
epochs = 4160                  # --epochs 5000
seed = 1                       # pick a trained seed in [1..10] per your run
model_type = "Diffusion"       # --model Diffusion (nickname if any was appended during training)

# Device config
cuda_index = 0                 # --device 0
DEVICE = torch.device(f"cuda:{cuda_index}" if torch.cuda.is_available() else "cpu")

# Data config
n_batches = 2
batch_size = 512
n_sample = None  # encoder ON in training; dataset returns variable set sizes
noise_sigma = 0.0  # optional: add noise to observed radius values

# Reproducibility for sampling/plotting
SET_SEED(42)

# Optional: set noise level used to generate observed radii (y)
set_circle_noise_sigma(noise_sigma)
print(f"Using device: {DEVICE}")


Using device: cuda:0


In [14]:
# Build a small dataloader and fetch one batch

dl, ds = return_circle_dl(n_batches=n_batches, batch_size=batch_size, n_sample=n_sample, return_ds=True)

theta_true, y_obs = next(iter(dl))  # theta_true: [B, 2], y_obs: [B, set_size, 1]
print("theta_true shape:", theta_true.shape)
print("y_obs shape:", y_obs.shape)

y_dim = y_obs.shape[-1]
x_dim = theta_true.shape[1]
print(f"y_dim={y_dim}, x_dim={x_dim}")


theta_true shape: torch.Size([512, 2])
y_obs shape: torch.Size([512, 960, 1])
y_dim=1, x_dim=2


In [15]:
# Reconstruct model architecture; must match training config
n_summaries = 256          # from main.py
num_hidden_layer = 4       # default training arg
use_encoder = True         # you trained with --use_encoder
sigma_data = 0.5           # you likely used default (unless --use_emperical_sigma was set)

model = DiffusionPosteriorSampler(
    y_dim=y_dim,
    x_dim=x_dim,
    n_summaries=n_summaries,
    num_hidden_layer=num_hidden_layer,
    device=DEVICE,
    use_encoder=use_encoder,
    data_type="iid",
    sigma_data=sigma_data,
).to(DEVICE)

# Load weights — filenames are: {load_path}/{epochs}_seed={seed}_{model_type}.pth
model = load_torch_model(model, load_path, epochs, seed, model_type)
_ = model.to(DEVICE).eval()

print("Model loaded and ready.")


Encoder is for iid data. If not, please check it.
Model loaded from /home/chu034/Yaohang_Li/cDiff/result/circle/4160_seed=1_Diffusion.pth
Model loaded and ready.


/home/chu034/Yaohang_Li/cDiff/utils.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(file_name))


In [16]:
# Run inference: posterior samples of theta given observed y

y_batch = y_obs.to(DEVICE)
with torch.no_grad():
    theta_est = model.sample(y_batch, num_steps=18)  # [B, 2]


In [17]:
theta_est

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], device='cuda:0')

In [ ]:

theta_est_cpu = theta_est.detach().cpu().numpy()
theta_true_cpu = theta_true.detach().cpu().numpy()

# Scatter: true theta vs estimated theta per dim
plt.figure(figsize=(10, 4))
for i in range(x_dim):
    plt.subplot(1, x_dim, i + 1)
    plt.scatter(theta_true_cpu[:, i], theta_est_cpu[:, i], s=10, alpha=0.5)
    plt.xlabel(f"True theta[{i}]")
    plt.ylabel(f"Est theta[{i}]")
    plt.title(f"Dim {i}")
plt.tight_layout()
plt.show()

# Optional: visualize samples for one observed set y
idx = 0
one_y = y_batch[idx:idx+1]
with torch.no_grad():
    samples = model.sample(one_y, num_steps=18)  # [1, 2]

# For visualization, draw multiple samples by repeating the same condition
with torch.no_grad():
    cond = one_y.repeat(200, 1, 1)  # 200 posterior samples
    samples_many = model.sample(cond, num_steps=18).detach().cpu().numpy()

plt.figure(figsize=(5, 5))
plt.scatter(samples_many[:, 0], samples_many[:, 1], s=10, alpha=0.6)
plt.xlabel("theta[0]")
plt.ylabel("theta[1]")
plt.title("Posterior samples for one observed set y")
plt.axis('equal')
plt.show()
